## More on visualisations in Altair

Last time we mentioned that the gallery of example visualisations https://altair-viz.github.io/gallery/index.html is a really helpful place to start with making a new visualisation. A good way of getting started is to cut and paste an example that is close to looking like a visualisation you want to generate and adapt it from there. 

If you haven't followed that link to look at some of the possible plots that can be made with Altair now is a good time to do that and just browse through to get a sense of what it can do and what the associated code looks like. 

This next section is a worked through example of using the gallery as a start point to develop a visualisation.

### Energy production multi-ring visualisation

This website has a great live dashboard giving details about power generation in the UK https://grid.iamkate.com. I really like this visualisation as it is a neat way of connecting the overall class of generation (fossil fuel/renewable/etc) with the exact breakdown in that class (gas/coal/solar/etc). We won't really deal with the dashboard aspect of this visualisation (it is really just some code behind the scenes that gets a feed of live data in a useable format, and then puts the visualisation on a live webpage), instead I've replicated a bit of the data in a static form in a spreadsheet. The task therefore is to recreate the visualisation in altair. 

First, let's import the modules we need, and load in our data and have a quick look at it. 

In [1]:
import pandas as pd
import altair as alt

energy = pd.read_excel('Energy generation over last year.xlsx')

print(energy.head)

<bound method NDFrame.head of                    Fuel  Generated value  Percentage       Energy type
0                  Coal             0.49         1.6       Fossil fuel
1  Gas (combined cycle)            12.06        39.9       Fossil fuel
2         Solar voltaic             1.22         4.0  Renewable energy
3                  Wind             5.79        19.1  Renewable energy
4         Hydroelectric             0.38         1.2  Renewable energy
5        Pumped storage             0.21         0.7      Other energy
6               Nuclear             4.94        16.3      Other energy
7               Biomass             2.18         7.2      Other energy
8                 Other             0.21         0.7      Other energy>


If we look at the sorts of visualisation on the webpage https://grid.iamkate.com they are done as round polar diagrams, partly a full pie chart style and then with an outer ring that is like a donut chart. 

Looking in the gallery we can find some details of how to produce a pie chart, https://altair-viz.github.io/gallery/pie_chart.html, so let's start there and just try plotting our data using the code from that page.

In [2]:
alt.Chart(energy).mark_arc().encode(
    theta=alt.Theta(field="Percentage", type="quantitative"),
    color=alt.Color(field="Fuel", type="nominal"),
)

alt.Chart(...)

That's a pretty good start, it makes a pie chart that makes sense and summarises our data. But immediately there are some things I would fix `field="Percentage", type="quantitative"` is a bit long-winded, we could replace it with `'Percentage:Q'`. Likewise for the colour we could write less and get the same result. I would also like to make the radius of the pie chart a bit smaller as it feels bigger than it need be, and bigger than the ones on the source website I am replicating. 

Lastly the source website had visualisations that are interactive, when I hover over them I see a summary of some of the information. We can get similar interactivity on our visualisation in altair (we'll discuss in more detail in a later notebook). To get something to appear when we hover we use a `tooltip`, and specify what fields in our data should be shown in the box on hovering over with the mouse. So, let's go ahead and add something of that kind too.

Putting those three sets of changes together we get this:

In [3]:
alt.Chart(energy).mark_arc(radius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Fuel:N'),
    tooltip=['Fuel','Generated value','Percentage']
)

alt.Chart(...)

So far so good, but we have focussed on the data that makes up the inner pie chart, next we need to think about the outer donut part of the visualisation. The data for that part is an aggregation (summarising if you prefer) of the data using the categories given in the column `'Energy type'`. Altair has various functions we can use called 'transforms' which allow you to plot summarised versions of your data in various ways (see https://altair-viz.github.io/user_guide/transform/index.html). Usefully for our purposes there is one called `transform_aggregate` that allows us to do precisely the thing we want to do and aggregate our different generation methods by adding them together depending on which `'Energy type'` they are.

Here's how we go about that for a pie chart. We make our pie chart using a new field which we can make up a new name for `'Overall Percentage'`. We then use our new aggregate function and need to define that new field as `sum(Percentage)` to sum up all the values in the `Percentage` field but aggregated by `'Energy type'` as noted by the `groupby=['Energy type']`.

Here's that working together:

In [4]:
alt.Chart(energy).mark_arc(radius=100).encode(
    theta=alt.Theta('Overall_Percentage:Q'),
    color=alt.Color('Energy type:N'),
).transform_aggregate(
    Overall_Percentage='sum(Percentage)',
    groupby=['Energy type']
)

alt.Chart(...)

Let's go a step further and add an interaction. Suppose I want to add the amount of energy generated in each `'Energy type'` as well (just as in the original I'm replicating), to do that I will need to define another new field in my aggregation. So let's do that, we can make a new field called `Overall_generated_value` and create it as `'sum(Generated value)` to get the amount of energy added up with that `'Energy type'`.
Lastly we add the new fields we calculated into the `tooltip` so that they appear when we hover the mouse over the plot.

In [5]:
alt.Chart(energy).mark_arc(radius=100).encode(
    theta=alt.Theta('Overall_Percentage:Q'),
    color=alt.Color('Energy type:N'),
    tooltip=['Energy type:N','Overall_generated_value:Q','Overall_Percentage:Q']
).transform_aggregate(
    Overall_Percentage='sum(Percentage)',
    Overall_generated_value='sum(Generated value)',
    groupby=['Energy type']
)

alt.Chart(...)

This is looking great for the sort of data and almost the right type of visualisation for the outer ring, but it is a full pie chart not a donut. So let's go back now to the gallery and look at the donut chart example in the gallery and try and replicate that with our data. The page for a donut chart is here https://altair-viz.github.io/gallery/donut_chart.html, so using our modifications from before (simplifying the field specifications, making the overall radius a bit smaller, and adding our interaction), we get:

In [6]:
alt.Chart(energy).mark_arc(innerRadius=50,radius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Fuel:N'),
    tooltip=['Fuel','Generated value','Percentage']
)

alt.Chart(...)

This looks good for a donut form of our main data (and it looks like the equivalent of our original pie chart).

Next we want to put the two parts we have been working on together. We have seen already we can do that by calculating both charts separately and then 'adding' them together to get them to appear on top of each other. The thing we need to do is just make sure that the radius of each part makes sense and fits together. But, doing that and putting together what we have to build this overall visualisation up, gives the following:

In [7]:
chart1 = alt.Chart(energy).mark_arc(radius=50).encode(
    theta=alt.Theta('Overall_Percentage:Q'),
    color=alt.Color('Energy type:N'),
    tooltip=['Energy type:N','Overall_generated_value:Q','Overall_Percentage:Q']
).transform_aggregate(
    Overall_Percentage='sum(Percentage)',
    Overall_generated_value='sum(Generated value)',
    groupby=['Energy type']
)

chart2 = alt.Chart(energy).mark_arc(innerRadius=50,radius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Fuel:N'),
    tooltip=['Fuel','Generated value','Percentage']
)

chart1+chart2

alt.LayerChart(...)

Oh no! This is almost there, but we can see something unexpected has happened - can you spot the issue? 

So, altair takes fields we mark as 'nominal' as containing words, basically as categories within that field of our data. For us `'Energy types'` and `'Fuel'` are both examples of this type of data. What altair does by default when it gets a field of that type is just sort them alphabetically, which when the ordering is not important (which is usually the case for nominal data) is a reasonable choice. Sadly here for us this has led to a problem, the inner and outer part of our chart don't line up as we would want them to (for example gas falls partly aligned with fossil fuel, but partly with other as well). This is just simply a result of both parts of the chart being arranged alphabetically.

How do we fix this? Well, for us the order of our nominal field is important. and so we need to specify the ordering. Here again is another point where we would look in the documentation to get some inspiration for how to go about this. The part of the documentation we might use to get into detail such as this is the API documentation here https://altair-viz.github.io/user_guide/API.html. This really details all of the different functions and options within them that we can use. If we look under `Encoding Channels` in the API documentation we can find a function that deals with the ordering of a field we are using in our chart https://altair-viz.github.io/user_guide/generated/channels/altair.Order.html. That documentation shows that we can specify for our field to be ordered by looking at a different field in the data. So that gives us a way out of our problem, we could make the outer donut have an extra order statement to specify that the plot should be ordered by the `'Energy type'` used in the inner plot. 

So, here goes, let's try that and see:

In [8]:
chart1 = alt.Chart(energy).mark_arc(radius=50).encode(
    theta=alt.Theta('Overall_Percentage:Q'),
    color=alt.Color('Energy type:N'),
    tooltip=['Energy type','Overall_generated_value:Q','Overall_Percentage:Q']
).transform_aggregate(
    Overall_Percentage='sum(Percentage)',
    Overall_generated_value='sum(Generated value)',
    groupby=['Energy type']
)

chart2 = alt.Chart(energy).mark_arc(innerRadius=50,radius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Fuel:N'),
    order=alt.Order('Energy type'),
    tooltip=['Fuel','Generated value','Percentage']
)

chart1+chart2

alt.LayerChart(...)

Finally that has sorted out our arrangement and ordering. We are very nearly there. So what else is left? 

I don't like the colour scheme here. So let's change that. We can do that by taking the color encoding and adding something altair calls a scale as part of the `color` part of our chart https://altair-viz.github.io/user_guide/generated/channels/altair.Color.html. Altair knows about a lot of colour schemes and they are listed here https://vega.github.io/vega/docs/schemes/#reference, I'll choose one with purples and oranges for no reason but that it is nice-looking! We can do that by adding `scale=alt.Scale(scheme='purpleorange')` to the `color` statement in the first of our defined charts.

Lastly the visualisation we are replicating has no legend with field names and colors listed, they just rely on the mouse hover interaction instead. For this split type of visualisation this might be a bit easier, and I would like to copy that feature, so for both `color` definitions with my two charts I will set `legend=None` which just turns off the legend being produced.

So at the end here is what we get:

In [9]:
chart1 = alt.Chart(energy).mark_arc(radius=100).encode(
    theta=alt.Theta('Overall_Percentage:Q'),
    color=alt.Color('Energy type:N',legend=None,scale=alt.Scale(scheme='purpleorange')),
    tooltip=['Energy type','Overall_generated_value:Q','Overall_Percentage:Q']
).transform_aggregate(
    Overall_Percentage='sum(Percentage)',
    Overall_generated_value='sum(Generated value)',
    groupby=['Energy type']
)

chart2 = alt.Chart(energy).mark_arc(innerRadius=50,radius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Fuel:N',legend=None),
    order=alt.Order('Energy type'),
    tooltip=['Fuel','Generated value','Percentage']
)

chart1+chart2

alt.LayerChart(...)

I'm now pretty happy with that, it isn't identical to the one in the original webpage, but it is close and overall copies the things I think are most compelling from that dashboard visualisation within our altair framework.

We've also seen some things about ordering fields, making interactions, and aggregation of our data in altair. 

Be aware that there would have been other ways to produce the same plot (indeed there will usually be more than one way to produce something - none of which typically is objectively 'right'). If you were thinking about using pandas to do the aggregation, or excel, and then producing the plot using two sources of data, indeed that would have been quite possible. I could have labelled the fields with `'a) Fossil fuels'` and so on to get the ordering to work properly by making things work alphabetically. Many reasonable solutions exist to the problems we thought about and worked through in developing this visualisation.


### Time-series data

A really important and commonly occuring type of data is where we get the series of something recorded over time (sometimes called a timeseries, time-series, longitudinal dataset). Having one of the fields in our dataset be related to time (time/date/year possibly) has some particular consequences, and altair has a special `temporal` data type specification which we haven't used yet at all. So, we should look at this type of data in some more detail. 

First let's open a dataset with some time-series aspect to it. We can use data about the rate of inflation from the Office for National Statistics (ONS) for the UK economy as an interesting one to start with https://www.ons.gov.uk/economy/inflationandpriceindices/datasets/consumerpriceinflation. 

In [10]:
inflation = pd.read_excel('inflation.xlsx')
print(inflation.head)

<bound method NDFrame.head of      Year     CPI
0    1800    13.5
1    1801    15.1
2    1802    11.6
3    1803    11.0
4    1804    11.3
..    ...     ...
216  2016  1037.7
217  2017  1074.9
218  2018  1110.8
219  2019  1139.3
220  2020  1156.4

[221 rows x 2 columns]>


So, this data set contains just over 200 years of inflation data for the UK! 

Let's make a first visualisation. The time field of this data is just restricted to year, so we could start by just pretending that this is an ordinal variable, and that will probably work pretty well. Here's how that looks:

In [11]:
alt.Chart(inflation).mark_bar(color='lightblue').encode(
    x='Year:O',
    y='CPI:Q',
)

alt.Chart(...)

That does indeed work alright. The plot is a bit long though, so we might want to change the width and height directly just to fit it in a bit better. These are properties attached to the chart that we can adjust with the properties function as follows:

In [12]:
alt.Chart(inflation).mark_bar(color='lightblue').encode(
    x='Year:O',
    y='CPI:Q',
).properties(
    width=700,
    height=500
)

alt.Chart(...)

Which is a better plot size, but now my axis labels are overlapping and ugly, but we can change an axis using the following specification to just list the axes labels that we want, we need to add something like `axis=alt.Axis(values=[1800,1850,1900,1950,2000])` to our x-axis specification where the list of values in the square brackets is the axis labels we want. Here goes:

In [13]:
alt.Chart(inflation).mark_bar(color='lightblue').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y='CPI:Q',
).properties(
    width=700,
    height=500
)

alt.Chart(...)

What else might we want to do with this? Well, another clear thing here is that the huge uptick in CPI over more recent years hides the changes in the relatively flat period from 1800-1950. One way of dealing with an axis where there is a large variability like this is to use a logarithmic scale (where we plot the logarithm of the value). Let's do that here and then we should be able to see some of the variation of the earlier part of the data.

In [14]:
alt.Chart(inflation).mark_line(color='lightblue').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
)

alt.Chart(...)

That's good now, and we have also moved back to a line as our mark to encode the data, certainly we can see a lot more of the variation in the curve early on.

We could now try and mark on some highlights on this chart to mark some major wars that the UK was engaged in. We might suppose that a war would have some effect on increasing inflation, so let's see. 

We can mark on a highlight as a red coloured line on top of the line already there for the ranges of each of the following wars (Crimea 1853-1856, Boer 1899-1902, WW1 1914-18, WW2 1939-45). This can be done by using a filter transfom to select only a few of the values along the x-axis when we make a red line. Using the fancy specification `alt.FieldRangePredicate(field='Year', range=[1853,1856])` we can select a range of values we want to include in our filter from a given filed 'Year' in this case.

In [15]:
mainplot= alt.Chart(inflation).mark_line(color='lightblue').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
)

highlight1 = alt.Chart(inflation).mark_line(color='red').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
).transform_filter( 
alt.FieldRangePredicate(field='Year', range=[1853,1856])
)

highlight2 = alt.Chart(inflation).mark_line(color='red').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
).transform_filter( 
alt.FieldRangePredicate(field='Year', range=[1899,1902])
)
highlight3 = alt.Chart(inflation).mark_line(color='red').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
).transform_filter( 
alt.FieldRangePredicate(field='Year', range=[1914,1918])
)

highlight4 = alt.Chart(inflation).mark_line(color='red').encode(
    x=alt.X('Year:O',axis=alt.Axis(values=[1800,1850,1900,1950,2000])),
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log")),
).properties(
    width=700,
    height=500
).transform_filter( 
alt.FieldRangePredicate(field='Year', range=[1939,1945])
)


mainplot+highlight1+highlight2+highlight3+highlight4

alt.LayerChart(...)

Again you can see the process of building up a more complex visualisation using parts that we add in step-by-step. From this it does indeed look like the periods of war lead to a generally quite steep increase in inflation, we'd probably need to see some more data to be really sure of our conclusion though.

We chose to specify our time axis as ordinal (i.e. discrete ordered numbers), but we could have also chosen quantitative as the years are just numbers essentially. Here's how that would look in one set of plots:

In [16]:
a = alt.Chart(inflation).mark_bar(color='lightblue').encode(
    x='Year:Q',
    y='CPI:Q',
)
b = alt.Chart(inflation).mark_line(color='lightblue').encode(
    x='Year:Q',
    y=alt.Y('CPI:Q',scale=alt.Scale(type="log"))
)
alt.hconcat(a,b)

alt.HConcatChart(...)

You can see that this works, but is maybe a bit less pretty in the way that it writes the labels for the x-axis.

Let's load in a second data set for UK inflation from the same source, but one focussed on the more recent past and with a more complicated field to specify time.

In [17]:
inflation2 = pd.read_excel('inflation2.xlsx')
inflation2.head()

,index date,name,CPI
0,198801,1988-01-01,48.395
1,198802,1988-02-01,48.551
2,198803,1988-03-01,48.717
3,198804,1988-04-01,49.280
4,198805,1988-05-01,49.529


You can see here two possibilities for using as an x-axis indexing this time-series. One is just an abstract code constructed by the data curator to try and encode the dates (in the field 'index date'). The second is an actual date encoded in 'YYYY-MM-DD' format  (in the field 'name'). Let's make some plots of both of these and try out altair's 'temporal' data type as well.

In [18]:
a = alt.Chart(inflation2).mark_line(color='lightblue').encode(
    x='name:T',
    y='CPI:Q',
)

b = alt.Chart(inflation2).mark_line(color='lightblue').encode(
    x='index date:Q',
    y='CPI:Q',
)

alt.hconcat(a,b)

alt.HConcatChart(...)

You can see the first visualisation on the left uses `'T'` to specify a temporal data type for the 'name' field. 

We could also try using ordinal as a data type for the 'index date'. Let's try it:

In [19]:
alt.Chart(inflation2).mark_line(color='lightblue').encode(
    x='index date:O',
    y='CPI:Q',
)

alt.Chart(...)

Again, a very wide plot. We could make this nicer by changing the size, and maybe labelling less on the x-axis. But you have seen those ideas already in this notebook so we won't repeat them here. 

Now we have seen the temporal data type `'T'` as the data type specification. The question then arises why could we not use that with our first inflation data set? It just had year as the relevant time field in our data, but that is also temporal. 

Well - let's try!

In [20]:
alt.Chart(inflation).mark_line(color='lightblue').encode(
    x='Year:T',
    y='CPI:Q',
)

alt.Chart(...)

Clearly that hasn't gone well. Altair has misinterpreted the years listed as perhaps times (we haved covered it here but the temporal data can hold time as well as dates).

We could however fix this if we really wanted to specify the temporal data type in our chart by using pandas. Pandas can convert a field in our data to a 'datetime'. Here's how that is done (this looks a bit technical, but don't forget you can look in the pandas documentation to understand this function, and you will find examples of similar things in code given online). 

In [21]:
#This replaces the Year field with a datetime version of the same data
inflation['Year'] = pd.to_datetime(inflation['Year'], format='%Y')  

alt.Chart(inflation).mark_line(color='lightblue').encode(
    x='Year:T',
    y='CPI:Q',
)

alt.Chart(...)

I'm sure you get from this, once again, the message that there are many ways to accomplish the same thing in python!

Lastly let's look at a more complex and nuanced dataset. This one is an NHS open dataset from [here](https://www.opendata.nhs.scot/dataset/mental-health-inpatient-activity/resource/83b38aa6-73c0-4489-b344-11081e7aa0ed), that details some aspects of mental health inpatient activity, and one of the fields within the data relates to year.  

In [22]:
mh = pd.read_csv('mental-health-inpatient-activity-age-sex.csv')
print(mh.head())

  FinancialYear        HBR HBRQF     AgeGroup     Sex Dataset DatasetQF  \
0       1997/98  S92000003     d   0-17 years    Male   Total         d   
1       1997/98  S92000003     d   0-17 years  Female   Total         d   
2       1997/98  S92000003     d  18-24 years    Male   Total         d   
3       1997/98  S92000003     d  18-24 years  Female   Total         d   
4       1997/98  S92000003     d  25-39 years    Male   Total         d   

   DischargeCount DischargeCountQF DischargeRates DischargeRatesQF  \
0             780                          133.93                    
1             710                          127.25                    
2            2020                          913.05                    
3            1310                          589.16                    
4            6450                         1123.62                    

   PatientsCount PatientCountQF PatientsRates PatientRatesQF  
0            700                       120.19                 
1 

This is a really big chunk of data that is quite complex. So the first thing we might do is simplify things a bit further. One of the data fields is 'HBR' which is the Health Board where the patient recorded is living. Each Health Board is an NHS administrative region of the UK, and we are in NHS Lothian which has code S08000024. So we could use pandas to match only the records in the dataset for which that value is in the 'HBR' field. Here's how that works - and we will make a new dataframe `mhlothian` with just these values:

In [23]:
mhlothian = mh[mh['HBR']=='S08000024']
print(mhlothian.head)

<bound method NDFrame.head of       FinancialYear        HBR HBRQF      AgeGroup     Sex Dataset DatasetQF  \
300         1997/98  S08000024          0-17 years    Male   Total         d   
301         1997/98  S08000024          0-17 years  Female   Total         d   
302         1997/98  S08000024         18-24 years    Male   Total         d   
303         1997/98  S08000024         18-24 years  Female   Total         d   
304         1997/98  S08000024         25-39 years    Male   Total         d   
...             ...        ...   ...           ...     ...     ...       ...   
13565       2020/21  S08000024         25-39 years  Female   SMR04             
13566       2020/21  S08000024         40-64 years    Male   SMR04             
13567       2020/21  S08000024         40-64 years  Female   SMR04             
13568       2020/21  S08000024        65plus years    Male   SMR04             
13569       2020/21  S08000024        65plus years  Female   SMR04             

       Di

Still complicated, but a bit smaller!

Let's start making some visualisations. We could start by just visualising the count of patients in each year - let's use our trick of just calling year an ordinal type of field:

In [24]:
alt.Chart(mhlothian).mark_bar(color='green').encode(
    alt.Y('PatientsCount:Q'),
    alt.X('FinancialYear:O'),
)

alt.Chart(...)

That's sort of worked, it did what we asked for, but look again at the data, for each year the number of patients is repeated as they are split by age group and then into data source. Let's look closer at just one year by doing our filtering again in pandas - but on year this time:

In [25]:
print(mhlothian[mhlothian['FinancialYear']=='1997/98'])

     FinancialYear        HBR HBRQF      AgeGroup     Sex Dataset DatasetQF  \
300        1997/98  S08000024          0-17 years    Male   Total         d   
301        1997/98  S08000024          0-17 years  Female   Total         d   
302        1997/98  S08000024         18-24 years    Male   Total         d   
303        1997/98  S08000024         18-24 years  Female   Total         d   
304        1997/98  S08000024         25-39 years    Male   Total         d   
305        1997/98  S08000024         25-39 years  Female   Total         d   
306        1997/98  S08000024         40-64 years    Male   Total         d   
307        1997/98  S08000024         40-64 years  Female   Total         d   
308        1997/98  S08000024        65plus years    Male   Total         d   
309        1997/98  S08000024        65plus years  Female   Total         d   
4860       1997/98  S08000024          0-17 years    Male   SMR01             
4861       1997/98  S08000024          0-17 years  F

So if we want to make a meaningful visualisation we will have to filter and aggregate our data some more. First let's filter to get just the 'Total' part of the field `'Dataset'` so that we have meaningful total patient number data that doesn't depend on the reporting mechanism.

Then taking that data let's use our `transform_aggregate` function in altair to group the data by year and make a time-series chart:

In [26]:
mhlothian_totals = mhlothian[mhlothian['Dataset']=='Total']

plt = alt.Chart(mhlothian_totals).encode(
    alt.Y('totPatients:Q'),
    alt.X('FinancialYear:N')
).transform_aggregate(
    totPatients='sum(PatientsCount)',
    groupby=['FinancialYear']
)

plt.mark_line(color='purple') + plt.mark_circle(size=100,color='pink')

alt.LayerChart(...)

One thing we often want to do is add a trend line to time-series plot. A straight line which is a fit to our data giving an idea of the trend of the data over time. We might like to do that here - and indeed altair has another transform function to help us to do it called `transform_regression()`. We could try it here:

In [27]:
plt = alt.Chart(mhlothian_totals).encode(
    alt.Y('totPatients:Q'),
    alt.X('FinancialYear:N')
).transform_aggregate(
    totPatients='sum(PatientsCount)',
    groupby=['FinancialYear']
)

plt.mark_point()+plt.transform_regression('FinancialYear','totPatients').mark_line(color='red')

alt.LayerChart(...)

Well, annoyingly that hasn't worked. This is because of the extra transformation in there performing the aggregation. Quite commonly using two of these transformations together fails - they don't tend to understand each other and play well together. 

So, what do we do if we still want the trendline? Well, we could do the aggregation using pandas instead of doing all the work in altair. So let's try that. Here's how we do that aggregation in pandas:

In [28]:
aggregated = mhlothian.groupby(['FinancialYear'],as_index=False).sum()
print(aggregated.head)

<bound method NDFrame.head of    FinancialYear  DischargeCount  PatientsCount
0        1997/98           12890           8370
1        1998/99           12680           8300
2        1999/00           12440           8340
3        2000/01           11820           7880
4        2001/02           12420           8090
5        2002/03           12670           8250
6        2003/04           11900           7950
7        2004/05           11850           7740
8        2005/06           11690           7660
9        2006/07           11120           7370
10       2007/08           11110           7500
11       2008/09           10710           7260
12       2009/10           10250           6950
13       2010/11           10290           7040
14       2011/12           10860           7320
15       2012/13           11170           7500
16       2013/14           10970           7210
17       2014/15           12060           7190
18       2015/16           12410           7480
19       2

That gives us our aggregated data, so now we can repeat our aggregated plot more simply without using a transform in altair to get the chart we made above:

In [29]:
plt = alt.Chart(aggregated).encode(
    alt.Y('PatientsCount:Q'),
    alt.X('FinancialYear:O'),
)

plt.mark_line(color='purple') + plt.mark_circle(size=100,color='pink')

alt.LayerChart(...)

Finally then, we can use the `transform_regression()` function to get a trend line for this data:

In [30]:
plt = alt.Chart(aggregated).encode(
    alt.X('FinancialYear:O'),
    alt.Y('PatientsCount:Q')
)

plt.transform_regression('FinancialYear','PatientsCount').mark_line(color='red')+plt.mark_point()

alt.LayerChart(...)

So, one last problem here as this didn't appear. Our x-axis is named dates in nominal data type. We can't make a regression line using this type of data, it needs to be quantitative in type. So to work around that we will need to use the line index as an x-axis and then use that for regression.

In [31]:
aggregated['index'] = aggregated.index

plt = alt.Chart(aggregated).encode(
    alt.X('index:Q'),
    alt.Y('PatientsCount:Q')
)

plt.transform_regression('index','PatientsCount').mark_line(color='red')+plt.mark_point()

alt.LayerChart(...)

Again we are nearly there. Lastly we might want to adjust the labels of the axes. This is a bit technical to do so we'll just present it here (if you need to do it yourself use this as a starting point to cut and paste and then edit). 

In [32]:
axis_labels = '''datum.label==0 ? '1997/98' : datum.label==1 ? "1998/99" : datum.label==2 ? "1999/00" : datum.label==3 ? "2000/01" : datum.label==4 ? "2001/02" : datum.label==5 ? "2002/03" : datum.label==6 ? "2003/04" : datum.label==7 ? "2004/05" : datum.label==8 ? "2005/06" : datum.label==9 ? "2006/07" : datum.label==10 ? "2007/08" : datum.label==11 ? "2008/09" : datum.label==12 ? "2009/10" : datum.label==13 ? "2010/11" : datum.label==14 ? "2011/12" : datum.label==15 ? "2012/13" : datum.label==16 ? "2013/14" : datum.label==17 ? "2014/15" : datum.label==18 ? "2015/16" : datum.label==19 ? "2016/17" : datum.label==20 ? "2017/18" : datum.label==21 ? "2018/19" : datum.label==22 ? "2019/20" :  datum.label==23 ? "2020/21": '2021/22' '''

plt = alt.Chart(aggregated).encode(
    alt.X('index:Q',title='Year',axis=alt.Axis(labelExpr=axis_labels,labelAngle=-90,tickCount=23),scale=alt.Scale(domain=(0, 23))),
    alt.Y('PatientsCount:Q',title='Patient Count')
)

plt.transform_regression('index','PatientsCount').mark_line(color='red')+plt.mark_point()

alt.LayerChart(...)

## Exercises

Any of the results or explorations you make of these exercises could be added to your portfolio that you need to submit for assessment. 

You could also try the tasks in Tableau.

1) There is a dataset for the UK debt over time that is available from the ONS again [here](https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicspending/bulletins/ukgovernmentdebtanddeficitforeurostatmaast/june2021). 

We have provided it in a nicer form in the excel file `Debt as proportion of GDP.xlsx`. Try making some visualisations of it. Could you add some highlights to note significant recent events (Gulf war 1990-1991, dot com crash and 9/11 attacks 2000-01, Iraq war 2003-2011) in a different colour.   

2) There is a interesting record of the trends in electricity generation in the UK available from the government open data portal [here](https://www.gov.uk/government/statistics/electricity-section-5-energy-trends). 

Again this is provided in `Energy generation trends 1.xlsx` and `Energy generation trends 2.xlsx` in slightly nicer forms (one is a flipped version of the other). Again, try opening these using pandas and make some visualisations of them. They contain categories in the same style as the first energy example, but listed over time so contain elements of the other time-series part of the notebook as well. There might therefore be lots of things you could try, but a first couple of ideas would be to look at how coal and solar have changed over time, maybe produce two pie charts to show the change in breakdown of energy production at the beginning and end of the timeseries. 

Note you have some missing data denoted with a `'[x]'` in the 'Solar' column, which you may need to handle.